In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

print(torch.__version__)
print(torch.cuda.is_available())

1.3.1
True


#### Check if is on CPU or GPU

```python
# Tensor
print(X.device)

# Model
print(next(model.parameters()).is_cuda)
```

# Train data by GPU

#### Sample Dataset

In [3]:
class RandomDataset(Dataset):
    def __init__(self, size, length):
        self.len = length
        self.data = torch.randn(length, size)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.len

In [4]:
# Create dataset: [100, 5]
N = 100
input_size = 5
random_dataset = RandomDataset(
    size=input_size,
    length=N)

# Dataloader: batch_size=30
batch_size = 30
rand_loader = DataLoader(
    dataset=random_dataset,
    batch_size=batch_size, shuffle=True)

In [5]:
print(f"Dataset shape: {random_dataset.data.size()}\n")
print(f"Dataloader length: {len(rand_loader)}")
for batch in rand_loader:
    print(batch.size())

Dataset shape: torch.Size([100, 5])

Dataloader length: 4
torch.Size([30, 5])
torch.Size([30, 5])
torch.Size([30, 5])
torch.Size([10, 5])


#### Sample model

In [6]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, use_GPU:bool=True, GPU_ids=[0]):
        super(Model, self).__init__()

        # Config
        self.use_GPU = use_GPU
        self.GPU_ids = GPU_ids

        # Layers
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, input):
        ## Hidden tensor
        batch_size = input.size()[0]

        if self.use_GPU:
            hidden_tensor = torch.randn(batch_size, input_size).cuda()
        else:
            hidden_tensor = torch.randn(batch_size, input_size)

        ## FW
        input = input + hidden_tensor
            # (batch_size, input_size) 
        output = self.fc(input)
            # (batch_size, output_size)

        ## Display Info
        gpu_id = torch.cuda.current_device()
        gpu_name = torch.cuda.get_device_name()
        print(f"=== Using GPU[{gpu_id}] {gpu_name} ===")
        print(f"input on {input.device}")
        print(f"hidden_tensor on {hidden_tensor.device}")
        print(f"output on {output.device}")

        return output

# Load to cpu

In [7]:
device = torch.device("cpu")
print(f"Use {device}")

Use cpu


In [8]:
# Load model
input_size = 5
output_size = 2
model = Model(input_size, output_size, use_GPU=False)
print(f"Is model using GPU: {next(model.parameters()).is_cuda}")

Is model using GPU: False


In [9]:
# Load data
for batch in rand_loader:
    # Load data
    Xb = batch

    # Train
    yb_ = model(Xb)
    print(f"Outside: Xb on {Xb.device}, yb_ on {yb_.device}\n")

=== Using GPU[0] Tesla K40m ===
input on cpu
hidden_tensor on cpu
output on cpu
Outside: Xb on cpu, yb_ on cpu

=== Using GPU[0] Tesla K40m ===
input on cpu
hidden_tensor on cpu
output on cpu
Outside: Xb on cpu, yb_ on cpu

=== Using GPU[0] Tesla K40m ===
input on cpu
hidden_tensor on cpu
output on cpu
Outside: Xb on cpu, yb_ on cpu

=== Using GPU[0] Tesla K40m ===
input on cpu
hidden_tensor on cpu
output on cpu
Outside: Xb on cpu, yb_ on cpu



# Load to 1 GPU

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Use {device}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

Use cuda
Number of GPUs: 4


In [11]:
# Load model
input_size = 5
output_size = 2
model = Model(input_size, output_size, use_GPU=True)
model.to(device)
print(f"Is model using GPU: {next(model.parameters()).is_cuda}")

Is model using GPU: True


In [12]:
# Load data
for batch in rand_loader:
    # Load data
    Xb = batch.to(device)

    # Train
    yb_ = model(Xb)
    print(f"Outside: Xb on {Xb.device}, yb_ on {yb_.device}\n")

=== Using GPU[0] Tesla K40m ===
input on cuda:0
hidden_tensor on cuda:0
output on cuda:0
Outside: Xb on cuda:0, yb_ on cuda:0

=== Using GPU[0] Tesla K40m ===
input on cuda:0
hidden_tensor on cuda:0
output on cuda:0
Outside: Xb on cuda:0, yb_ on cuda:0

=== Using GPU[0] Tesla K40m ===
input on cuda:0
hidden_tensor on cuda:0
output on cuda:0
Outside: Xb on cuda:0, yb_ on cuda:0

=== Using GPU[0] Tesla K40m ===
input on cuda:0
hidden_tensor on cuda:0
output on cuda:0
Outside: Xb on cuda:0, yb_ on cuda:0



#### Save model on 1 GPU

In [13]:
# Check params
model.state_dict()

OrderedDict([('fc.weight',
              tensor([[ 0.3728,  0.3349, -0.3626, -0.0367,  0.1534],
                      [-0.2738, -0.2812,  0.4316,  0.2844,  0.2578]], device='cuda:0')),
             ('fc.bias', tensor([-0.0507,  0.3611], device='cuda:0'))])

In [14]:
# Save model
torch.save(model.state_dict(), 'mymodel.pt')

In [15]:
# Load model
model_load = Model(input_size, output_size)
model_load.load_state_dict(torch.load('mymodel.pt'))

<All keys matched successfully>